# Ejercicio 3: Preprocesamiento

## Objetivo de la práctica

1. Comprender y aplicar normalización, tokenización, stopwords, stemming y n-gramas.
2. Medir el impacto de cada paso en el vocabulario y los tokens.

#### 0. Cargar el Corpus

Vamos a trabajar con el corpus de Movie Reviews de IMDB

In [3]:
import pandas as pd

In [4]:
path = '/kaggle/input/imdbdatasetzip/IMDB Dataset.csv'
df = pd.read_csv(path, encoding='utf-8')
print(df.head())
print(len(df))

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
50000


#### 1. Limpieza 

Limpiar los documentos de caracteres que no corresponden

In [5]:
# Limpieza extendida (recomendado)
import re

# compilar patrones una sola vez
TAG_RE    = re.compile(r'<.*?>')          # quita tags HTML simples
CTRL_RE   = re.compile(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]')  # chars de control
PUNCT_RE  = re.compile(r'[^A-Za-z0-9\s]') # todo lo que NO sea letra / número / espacio -> quitar
MULTI_WS  = re.compile(r'\s+')

def clean_text_extended(text):
    """Quita HTML, caracteres de control, signos/puntuación, y colapsa espacios.
       Devuelve string limpio; si text no es str, devuelve empty string.
    """
    if not isinstance(text, str):
        return ""
    t = text
    # 1) quitar tags HTML (reemplazo por espacio)
    t = TAG_RE.sub(" ", t)
    # 2) quitar caracteres de control (incluye \x08)
    t = CTRL_RE.sub("", t)
    # 3) quitar puntuación y símbolos no alfanuméricos (quita \, (, ), ., ,, ', ", ?, !, -, etc)
    t = PUNCT_RE.sub(" ", t)
    # 4) colapsar espacios múltiples y trim
    t = MULTI_WS.sub(" ", t).strip()
    return t


In [6]:
# guardar el review original
df['review_original'] = df['review']

# aplicar la limpieza extendida
df['review_clean'] = df['review'].apply(clean_text_extended)

# comprobar cuántas filas cambiaron
changed_mask = df['review'] != df['review_clean']
print("Filas totales:", len(df))
print("Filas cambiadas:", changed_mask.sum())


Filas totales: 50000
Filas cambiadas: 49996


In [7]:
# inspeccion de un elemento en especifico que tenia caracteres de control
# se usa repr para visualizar los caracteres de control.
idx = 14497
print("\n=== ORIGINAL ===")
print(repr(df['review_original'].iloc[idx][:300])) #se muestran los primeros 300 caracteres con [:300] en un string
print("\n=== LIMPIO ===")
print(repr(df['review_clean'].iloc[idx][:300]))


=== ORIGINAL ===
"\x08\x08\x08\x08A Turkish Bath sequence in a film noir located in New York in the 50's, that must be a hint at something ! Something that curiously, in all the previous comments, no one has pointed out , but seems to me essential to the understanding of this movie <br /><br />the Turkish Baths sequence: a back "

=== LIMPIO ===
'A Turkish Bath sequence in a film noir located in New York in the 50 s that must be a hint at something Something that curiously in all the previous comments no one has pointed out but seems to me essential to the understanding of this movie the Turkish Baths sequence a back street at night the entr'


#### 2. Normalización

Convertir todos los tokens a minúsculas.

Elimina puntuación y símbolos no alfabéticos.

In [10]:
# Convertir a minúsculas + tokenizar
# Ya se eliminó la puntuación en el paso anterior al quedarnos solo con lo que queremos (letras y números)
# Aquí se eliminan los números puesto que son símbolos no alfabéticos

def normalize_text(text):
    if not isinstance(text, str):
        return []
    # minúsculas
    text = text.lower()
    # tokenizar por espacios
    tokens = text.split()
    # eliminar tokens no alfabéticos (números)
    tokens = [t for t in tokens if re.fullmatch(r'[a-z]+', t)]
    return tokens


In [11]:
# se crea la columna tokens_norm en el dataframe
df['tokens_norm'] = df['review_clean'].apply(normalize_text)

In [12]:
# Comparativa de la evolución del texto a tokens
print("\n=== Texto Original ===")
print(repr(df['review'].iloc[14497][:100]))
print("\n=== Tokens Resultantes ===")
print(df['tokens_norm'].iloc[14497][:15])


=== Texto Original ===
"\x08\x08\x08\x08A Turkish Bath sequence in a film noir located in New York in the 50's, that must be a hint at s"

=== Tokens Resultantes ===
['a', 'turkish', 'bath', 'sequence', 'in', 'a', 'film', 'noir', 'located', 'in', 'new', 'york', 'in', 'the', 's']


#### 3. Eliminación de Stopwords

Eliminar las palabras vacías (stopwords) usando una lista estándar de la librería _nltk_.

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stopwords_en = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
# Definicion de función que elimina los stopwords en inglés
def remove_stopwords(tokens):
    return [t for t in tokens if t not in stopwords_en]

In [15]:
# Se crea la columna de tokens sin Stopwords
df['tokens_no_stop'] = df['tokens_norm'].apply(remove_stopwords)

In [16]:
# Se verifica la cantidad de tokens inicial y final, luego de aplicar la remoción de Stopwords
idx = 14497
print("\n=== Cantidad de tokens normalizados ===")
print(len(df['tokens_norm'].iloc[idx]))
print("\n=== Cantidad de tokens sin Stopwords ===")
print(len(df['tokens_no_stop'].iloc[idx]))
print("\nSe ha reducido casi a la mitad la cantidad de tokens")


=== Cantidad de tokens normalizados ===
619

=== Cantidad de tokens sin Stopwords ===
313

Se ha reducido casi a la mitad la cantidad de tokens


#### 4. Stemming 

Reducir el espacio de palabras

In [17]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [18]:
# Definición de la función que aplica stemming a una lista de tokens
def apply_stemming(tokens):
    """Recibe lista de tokens (strings) -> devuelve lista de stems"""
    if not isinstance(tokens, (list, tuple)):
        return []
    return [stemmer.stem(t) for t in tokens]

In [19]:
# Se aplica y se guarda en nueva columna del dataframe
df['tokens_stemmed'] = df['tokens_no_stop'].apply(apply_stemming)

In [20]:
# se visualiza una muestra de los tokens antes y después del stemming
idx = 14497
print("\n=== Muestra de Tokens sin Stopwords ===")
print(df['tokens_no_stop'].iloc[idx][:30])
print("\n=== Muestra de Tokens Stemmed ===")
print(df['tokens_stemmed'].iloc[idx][:30])
print("\nNo se reduce la cantidad de palabras, pero las que se recortan, se reducen hasta su raíz.")


=== Muestra de Tokens sin Stopwords ===
['turkish', 'bath', 'sequence', 'film', 'noir', 'located', 'new', 'york', 'must', 'hint', 'something', 'something', 'curiously', 'previous', 'comments', 'one', 'pointed', 'seems', 'essential', 'understanding', 'movie', 'turkish', 'baths', 'sequence', 'back', 'street', 'night', 'entrance', 'sleazy', 'sauna']

=== Muestra de Tokens Stemmed ===
['turkish', 'bath', 'sequenc', 'film', 'noir', 'locat', 'new', 'york', 'must', 'hint', 'someth', 'someth', 'curious', 'previou', 'comment', 'one', 'point', 'seem', 'essenti', 'understand', 'movi', 'turkish', 'bath', 'sequenc', 'back', 'street', 'night', 'entranc', 'sleazi', 'sauna']

No se reduce la cantidad de palabras, pero las que se recortan, se reducen hasta su raíz.


#### 5. Verificar la diferencia

Comparar el tamaño del diccionario de términos del corpus antes y después de aplicar el preprocesamiento 

In [26]:
def vocab_from_series_of_strings(series):
    """
    Recibe una columna/iterable donde cada elemento es un string (texto).
    Devuelve un set con todas las palabras únicas (separadas por espacios).
    """
    vocab = set()                      # conjunto vacío donde acumulamos tipos únicos
    for text in series:
        if not isinstance(text, str):  # si la fila no es string (NaN, None, número) la saltamos
            continue
        words = text.split()           # separación simple por espacios
        vocab.update(words)            # añade todas las palabras (ignora duplicados)
    return vocab


def vocab_from_series_of_tokenlists(series):
    """
    Recibe una columna/iterable donde cada elemento es una lista (o tupla) de tokens.
    Devuelve un set con todos los tokens únicos.
    Si alguna fila es string en lugar de lista, la tokeniza por espacios para manejarlo.
    """
    vocab = set()
    for item in series:
        # si ya es lista/tupla se itera directamente
        if isinstance(item, (list, tuple)):
            vocab.update(item)
        # si es string, lo tokenizamos por espacios (soporte robusto)
        elif isinstance(item, str):
            vocab.update(item.split())
        # si es otro tipo (NaN, None, número) se ignora
    return vocab


In [41]:
# 1) Raw vocab (word forms tal cual del original, tokenizados por espacios)
raw_vocab = vocab_from_series_of_strings(df['review_original'])

# 2) Clean vocab (texto limpio, antes de normalizar a minúsculas o quitar números)
clean_vocab = vocab_from_series_of_strings(df['review_clean'])

# 3) Normalized vocab (tokens_norm es lista de tokens ya normalizados y sin números)
norm_vocab = vocab_from_series_of_tokenlists(df['tokens_norm'])

# 4) No-stop vocab
no_stop_vocab = vocab_from_series_of_tokenlists(df['tokens_no_stop'])

# 5) Stemmed vocab
stemmed_vocab = vocab_from_series_of_tokenlists(df['tokens_stemmed'])

# Mostrar resultados
etapasVocab = {
    'raw': raw_vocab,
    'clean': clean_vocab,
    'normalized': norm_vocab,
    'no_stopwords': no_stop_vocab,
    'stemmed': stemmed_vocab
}

for name, vocab in etapasVocab.items():
    print(f"vocab. {name:13s}: {len(vocab):7d} palabras")

vocab. raw          :  438729 palabras
vocab. clean        :  129426 palabras
vocab. normalized   :   99319 palabras
vocab. no_stopwords :   99166 palabras
vocab. stemmed      :   68921 palabras


In [30]:
def reduction(a, b):
    diferencia = len(a) - len(b)
    if len(a) > 0:
        porcentaje = diferencia * 100.0/len(a)
    else:
        porcentaje = 0.0
        
    porcentaje_str = f"{porcentaje:.2f} %"
    return diferencia, porcentaje_str

In [25]:
# Comparación de los tamaños de diccionario antes y después del preprocesamiento en cada etapa
# se representa (cantPalabras, porcentajeDeReducción)
print("raw -> clean:", reduction(raw_vocab, clean_vocab))
print("clean -> normalized:", reduction(clean_vocab, norm_vocab))
print("normalized -> no_stopwords:", reduction(norm_vocab, no_stop_vocab))
print("no_stopwords -> stemmed:", reduction(no_stop_vocab, stemmed_vocab))
print("raw -> stemmed:", reduction(raw_vocab, stemmed_vocab))

raw -> clean: (309303, '70.50 %')
clean -> normalized: (30107, '23.26 %')
normalized -> no_stopwords: (153, '0.15 %')
no_stopwords -> stemmed: (30245, '30.50 %')
raw -> stemmed: (369808, '84.29 %')
